In [5]:
import pandas as pd
import numpy as np

# Baseline

In [ ]:
# Rainfall Forecasting using Recurrent Neural Networks (RNN-LSTM)
# Berdasarkan paper: Prasetya & Djamal (2019)

# ============================================================================
# 1. IMPORT LIBRARIES
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

# Set random seed untuk reproducibility
np.random.seed(42)
keras.utils.set_random_seed(42)

print("Library berhasil diimport!")

# ============================================================================
# 2. LOAD DATA
# ============================================================================
# Ganti dengan path file Anda
df = pd.read_excel('updated_dataset_final.xlsx')


# UNCOMMENT baris berikut dan sesuaikan path file:
df = pd.read_excel('updated_dataset_final.xlsx')
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df = df.sort_values('Tanggal').reset_index(drop=True)

print("Data berhasil dimuat!")
# print(f"Jumlah data: {len(df)}")
# print(f"\nInfo data:\n{df.info()}")
# print(f"\nSample data:\n{df.head()}")

# ============================================================================
# 3. PREPROCESSING DATA
# ============================================================================

def interpolate_missing_data(df):
    """
    Menangani missing data dengan interpolasi linear
    """
    df_clean = df.copy()
    
    # Interpolasi untuk kolom numerik
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_clean[col] = df_clean[col].interpolate(method='linear', limit_direction='both')
    
    return df_clean

def extract_weekly_features(df):
    """
    Mengekstrak fitur mingguan dengan mengambil nilai maksimum setiap minggu
    Sesuai dengan formula di paper: x = Σ(n/k) * max(xi, x_i+1+7)
    """
    df_weekly = df.copy()
    df_weekly['Week'] = df_weekly['Tanggal'].dt.isocalendar().week
    df_weekly['Year'] = df_weekly['Tanggal'].dt.year
    
    # Agregasi mingguan - ambil nilai maksimum
    weekly_data = df_weekly.groupby(['Year', 'Week']).agg({
        'Temperatur Minimum': 'max',
        'Temperatur Maksimum': 'max',
        'Temperatur Rata-rata': 'max',
        'Kelembapan Rata-rata': 'max',
        'Curah Hujan (mm)': 'max',
        'Lamanya Penyinaran Matahari': 'max',
        'Kecepatan Angin Maksimum': 'max',
        'Kecepatan Angin Rata-rata': 'max',
        'Tanggal': 'last'
    }).reset_index()
    
    return weekly_data

def normalize_data(df, feature_cols):
    """
    Normalisasi data ke rentang [0, 1]
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_normalized = df.copy()
    df_normalized[feature_cols] = scaler.fit_transform(df[feature_cols])
    
    return df_normalized, scaler

def classify_rainfall(rainfall):
    """
    Klasifikasi curah hujan menjadi 5 kelas:
    0: Very Low (<5mm)
    1: Low (5-20mm)
    2: Medium (20-50mm)
    3: High (50-100mm)
    4: Very High (>100mm)
    """
    if rainfall < 5:
        return 0
    elif rainfall < 20:
        return 1
    elif rainfall < 50:
        return 2
    elif rainfall < 100:
        return 3
    else:
        return 4

# Uncomment untuk menjalankan preprocessing:

# 3.1 Interpolasi missing data
df = interpolate_missing_data(df)
print("Missing data berhasil diinterpolasi!")

# 3.2 Ekstraksi fitur mingguan
df_weekly = extract_weekly_features(df)
print(f"Data mingguan berhasil diekstrak! Total: {len(df_weekly)} minggu")

# 3.3 Klasifikasi curah hujan
df_weekly['Rainfall_Class'] = df_weekly['Curah Hujan (mm)'].apply(classify_rainfall)

# 3.4 Normalisasi
feature_cols = ['Temperatur Minimum', 'Temperatur Maksimum', 'Temperatur Rata-rata',
                'Kelembapan Rata-rata', 'Curah Hujan (mm)', 'Lamanya Penyinaran Matahari',
                'Kecepatan Angin Maksimum', 'Kecepatan Angin Rata-rata']

df_normalized, scaler = normalize_data(df_weekly, feature_cols)
print("Data berhasil dinormalisasi!")


# ============================================================================
# 4. PREPARE SEQUENCES (OVERLAPPING WINDOWS)
# ============================================================================

def create_sequences(data, feature_cols, target_col, sequence_length=52):
    """
    Membuat sequences dengan overlap untuk training RNN
    Setiap sequence berisi data 1 tahun (52 minggu)
    
    Parameters:
    - data: DataFrame yang sudah dinormalisasi
    - feature_cols: list kolom fitur
    - target_col: kolom target (Rainfall_Class)
    - sequence_length: panjang sequence (default 52 minggu = 1 tahun)
    
    Returns:
    - X: array of sequences (samples, timesteps, features)
    - y: array of targets
    """
    X, y = [], []
    
    for i in range(len(data) - sequence_length):
        # Ambil sequence 52 minggu
        sequence = data[feature_cols].iloc[i:i+sequence_length].values
        # Target adalah minggu ke-53 (prediksi minggu depan)
        target = data[target_col].iloc[i+sequence_length]
        
        X.append(sequence)
        y.append(target)
    
    return np.array(X), np.array(y)

# Uncomment untuk membuat sequences:
"""
sequence_length = 52  # 1 tahun = 52 minggu

X, y = create_sequences(df_normalized, feature_cols, 'Rainfall_Class', sequence_length)

print(f"Shape X: {X.shape}")  # (samples, 52, n_features)
print(f"Shape y: {y.shape}")  # (samples,)
print(f"Total sequences: {len(X)}")

# Visualisasi distribusi kelas
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.hist(y, bins=5, edgecolor='black')
plt.xlabel('Rainfall Class')
plt.ylabel('Frequency')
plt.title('Distribution of Rainfall Classes')
plt.xticks([0, 1, 2, 3, 4], ['Very Low', 'Low', 'Medium', 'High', 'Very High'])

plt.subplot(1, 2, 2)
class_names = ['Very Low\n(<5mm)', 'Low\n(5-20mm)', 'Medium\n(20-50mm)', 
               'High\n(50-100mm)', 'Very High\n(>100mm)']
unique, counts = np.unique(y, return_counts=True)
plt.bar(unique, counts, edgecolor='black')
plt.xlabel('Rainfall Class')
plt.ylabel('Count')
plt.title('Rainfall Class Distribution')
plt.xticks(unique, class_names, rotation=45, ha='right')
plt.tight_layout()
plt.show()
"""

# ============================================================================
# 5. SPLIT DATA (80% TRAINING, 20% TESTING)
# ============================================================================

# Uncomment untuk split data:
"""
# Split data sesuai paper: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Convert target ke categorical (one-hot encoding)
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)
"""

# ============================================================================
# 6. BUILD RNN-LSTM MODEL
# ============================================================================

def build_lstm_model(input_shape, num_classes=5, lstm_units=64, dropout_rate=0.2):
    """
    Membangun model RNN-LSTM sesuai paper
    
    Architecture:
    - Input Layer: 64 neurons
    - Hidden Layer (LSTM): 64 neurons
    - Dropout: 0.2
    - Dense Layer: 13 neurons
    - Output Layer: 5 classes (softmax)
    """
    model = Sequential([
        # Input + LSTM Layer (64 units)
        LSTM(lstm_units, input_shape=input_shape, return_sequences=False),
        
        # Dropout untuk mencegah overfitting
        Dropout(dropout_rate),
        
        # Dense Layer (13 neurons dengan ReLU activation)
        Dense(13, activation='relu'),
        
        # Output Layer (5 classes dengan Sigmoid activation)
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Uncomment untuk build model:
"""
# Input shape: (sequence_length, n_features)
input_shape = (X_train.shape[1], X_train.shape[2])

model = build_lstm_model(input_shape, num_classes=5, lstm_units=64, dropout_rate=0.2)

# Print model summary
model.summary()
"""

# ============================================================================
# 7. COMPILE MODEL
# ============================================================================

# Uncomment untuk compile model:
"""
# Sesuai paper: menggunakan Adam optimizer dengan learning rate 0.001
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model berhasil dikompilasi!")
"""

# ============================================================================
# 8. TRAINING MODEL
# ============================================================================

# Uncomment untuk training:
"""
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_rainfall_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Training dengan 500 epochs sesuai paper
history = model.fit(
    X_train, y_train_cat,
    epochs=500,
    batch_size=32,
    validation_data=(X_test, y_test_cat),
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

print("Training selesai!")
"""

# ============================================================================
# 9. VISUALISASI HASIL TRAINING
# ============================================================================

def plot_training_history(history):
    """
    Visualisasi accuracy dan loss selama training
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot Accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # Plot Loss
    axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Uncomment untuk visualisasi:
"""
plot_training_history(history)
"""

# ============================================================================
# 10. EVALUASI MODEL
# ============================================================================

# Uncomment untuk evaluasi:
"""
# Evaluasi pada training data
train_loss, train_accuracy = model.evaluate(X_train, y_train_cat, verbose=0)
print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Training Loss: {train_loss:.4f}")

# Evaluasi pada test data
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# Prediksi
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification Report
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
class_names = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
"""

# ============================================================================
# 11. COMPARISON: RNN vs CNN (OPSIONAL)
# ============================================================================

def build_cnn_model(input_shape, num_classes=5):
    """
    Build 1D CNN model untuk perbandingan
    """
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
    
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Uncomment untuk perbandingan RNN vs CNN:
"""
print("\n" + "="*60)
print("PERBANDINGAN RNN vs CNN")
print("="*60)

# Build CNN model
cnn_model = build_cnn_model(input_shape, num_classes=5)
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train CNN
history_cnn = cnn_model.fit(
    X_train, y_train_cat,
    epochs=500,
    batch_size=32,
    validation_data=(X_test, y_test_cat),
    callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
    verbose=0
)

# Evaluate CNN
cnn_train_loss, cnn_train_acc = cnn_model.evaluate(X_train, y_train_cat, verbose=0)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test, y_test_cat, verbose=0)

# Comparison Table
comparison_df = pd.DataFrame({
    'Method': ['RNN (LSTM)', 'CNN (1D)'],
    'Training Accuracy (%)': [train_accuracy*100, cnn_train_acc*100],
    'Training Loss': [train_loss, cnn_train_loss],
    'Test Accuracy (%)': [test_accuracy*100, cnn_test_acc*100],
    'Test Loss': [test_loss, cnn_test_loss]
})

print(comparison_df.to_string(index=False))
"""

# ============================================================================
# 12. COMPARISON: SGD vs ADAM OPTIMIZER (OPSIONAL)
# ============================================================================

# Uncomment untuk perbandingan SGD vs Adam:
"""
print("\n" + "="*60)
print("PERBANDINGAN SGD vs ADAM OPTIMIZER")
print("="*60)

# Build model dengan SGD
model_sgd = build_lstm_model(input_shape, num_classes=5)
model_sgd.compile(
    optimizer=SGD(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train dengan SGD
history_sgd = model_sgd.fit(
    X_train, y_train_cat,
    epochs=500,
    batch_size=32,
    validation_data=(X_test, y_test_cat),
    callbacks=[EarlyStopping(patience=50, restore_best_weights=True)],
    verbose=0
)

# Evaluate SGD
sgd_train_loss, sgd_train_acc = model_sgd.evaluate(X_train, y_train_cat, verbose=0)
sgd_test_loss, sgd_test_acc = model_sgd.evaluate(X_test, y_test_cat, verbose=0)

# Comparison Table
optimizer_comparison = pd.DataFrame({
    'Optimizer': ['Adam', 'SGD'],
    'Training Accuracy (%)': [train_accuracy*100, sgd_train_acc*100],
    'Training Loss': [train_loss, sgd_train_loss],
    'Test Accuracy (%)': [test_accuracy*100, sgd_test_acc*100],
    'Test Loss': [test_loss, sgd_test_loss]
})

print(optimizer_comparison.to_string(index=False))
"""

# ============================================================================
# 13. TESTING DIFFERENT LEARNING RATES (OPSIONAL)
# ============================================================================

# Uncomment untuk testing learning rates:
"""
print("\n" + "="*60)
print("TESTING DIFFERENT LEARNING RATES")
print("="*60)

learning_rates = [0.001, 0.002, 0.010, 0.040, 0.100, 0.400, 0.600, 0.800]
lr_results = []

for lr in learning_rates:
    print(f"\nTesting learning rate: {lr}")
    
    # Build and compile model
    model_lr = build_lstm_model(input_shape, num_classes=5)
    model_lr.compile(
        optimizer=Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train
    history_lr = model_lr.fit(
        X_train, y_train_cat,
        epochs=500,
        batch_size=32,
        validation_data=(X_test, y_test_cat),
        callbacks=[EarlyStopping(patience=30, restore_best_weights=True)],
        verbose=0
    )
    
    # Evaluate
    train_loss_lr, train_acc_lr = model_lr.evaluate(X_train, y_train_cat, verbose=0)
    test_loss_lr, test_acc_lr = model_lr.evaluate(X_test, y_test_cat, verbose=0)
    
    lr_results.append({
        'Learning Rate': lr,
        'Training Accuracy (%)': train_acc_lr*100,
        'Training Loss': train_loss_lr,
        'Test Accuracy (%)': test_acc_lr*100,
        'Test Loss': test_loss_lr
    })

lr_df = pd.DataFrame(lr_results)
print("\n" + lr_df.to_string(index=False))

# Visualize learning rate comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(lr_df['Learning Rate'], lr_df['Training Accuracy (%)'], 
             marker='o', label='Training', linewidth=2)
axes[0].plot(lr_df['Learning Rate'], lr_df['Test Accuracy (%)'], 
             marker='s', label='Test', linewidth=2)
axes[0].set_xlabel('Learning Rate', fontsize=12)
axes[0].set_ylabel('Accuracy (%)', fontsize=12)
axes[0].set_title('Accuracy vs Learning Rate', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')

axes[1].plot(lr_df['Learning Rate'], lr_df['Training Loss'], 
             marker='o', label='Training', linewidth=2)
axes[1].plot(lr_df['Learning Rate'], lr_df['Test Loss'], 
             marker='s', label='Test', linewidth=2)
axes[1].set_xlabel('Learning Rate', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Loss vs Learning Rate', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')

plt.tight_layout()
plt.show()
"""

# ============================================================================
# 14. PREDICTION FUNCTION
# ============================================================================

def predict_rainfall(model, input_sequence, scaler):
    """
    Fungsi untuk memprediksi curah hujan minggu depan
    
    Parameters:
    - model: trained model
    - input_sequence: sequence data 52 minggu terakhir (normalized)
    - scaler: scaler yang digunakan untuk normalisasi
    
    Returns:
    - predicted_class: kelas prediksi (0-4)
    - class_name: nama kelas
    - confidence: confidence score
    """
    # Reshape untuk prediksi
    input_seq = input_sequence.reshape(1, input_sequence.shape[0], input_sequence.shape[1])
    
    # Prediksi
    prediction_probs = model.predict(input_seq, verbose=0)
    predicted_class = np.argmax(prediction_probs, axis=1)[0]
    confidence = prediction_probs[0][predicted_class] * 100
    
    # Mapping kelas ke nama
    class_mapping = {
        0: 'Very Low (<5mm)',
        1: 'Low (5-20mm)',
        2: 'Medium (20-50mm)',
        3: 'High (50-100mm)',
        4: 'Very High (>100mm)'
    }
    
    class_name = class_mapping[predicted_class]
    
    return predicted_class, class_name, confidence

# Uncomment untuk testing prediksi:
"""
# Contoh prediksi
sample_sequence = X_test[0]
pred_class, pred_name, pred_confidence = predict_rainfall(model, sample_sequence, scaler)

print(f"\nPrediksi Curah Hujan Minggu Depan:")
print(f"Kelas: {pred_name}")
print(f"Confidence: {pred_confidence:.2f}%")
print(f"Actual: {class_mapping[y_test[0]]}")
"""

# ============================================================================
# 15. SAVE AND LOAD MODEL
# ============================================================================

# Uncomment untuk save model:
"""
# Save model
model.save('rainfall_forecast_model.h5')
print("Model berhasil disimpan!")

# Save scaler
import joblib
joblib.dump(scaler, 'scaler.pkl')
print("Scaler berhasil disimpan!")
"""

# Uncomment untuk load model:
"""
# Load model
from tensorflow.keras.models import load_model
loaded_model = load_model('rainfall_forecast_model.h5')

# Load scaler
loaded_scaler = joblib.load('scaler.pkl')

print("Model dan scaler berhasil dimuat!")
"""

print("\n" + "="*60)
print("SCRIPT SELESAI!")
print("="*60)
print("\nCatatan:")
print("- Uncomment bagian-bagian kode sesuai kebutuhan")
print("- Pastikan data sudah dimuat dengan benar sebelum menjalankan")
print("- Sesuaikan path file untuk load dan save data/model")
print("="*60)

Library berhasil diimport!
Data berhasil dimuat!

SCRIPT SELESAI!

Catatan:
- Uncomment bagian-bagian kode sesuai kebutuhan
- Pastikan data sudah dimuat dengan benar sebelum menjalankan
- Sesuaikan path file untuk load dan save data/model
